In [ ]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import date
import schedule
import time

In [ ]:
def updateMenu():
    todayMenu = requests.request('GET','https://front.cjfreshmeal.co.kr/meal/v1/today-all-meal?storeIdx=6442')
    todayMenuLunch = json.loads(todayMenu.text)['data']['2']
    menu = [0] * 5
    for i in range(5):
        menu[i] = {
            'name':todayMenuLunch[i]['name'],
            'side':todayMenuLunch[i]['side'],
            'corner':todayMenuLunch[i]['corner'],
            'kcal':todayMenuLunch[i]['kcal'],
            'thumbnailurl':todayMenuLunch[i]['thumbnailUrl']
        }
    return menu

In [ ]:
def getRenderString(menu):
    renderstring = ''
    renderstring += '# ' + date.today().strftime("%Y-%m-%d") + ' 오늘의 점심\n'
    for i in range(5):
        renderstring += '## ' + str(i+1) + '. ' + menu[i]['corner'] + '\n'
        renderstring += '### ' + '**' + menu[i]['name'] + '**\n'
        renderstring += menu[i]['side'] + '\n'
        renderstring += str(int(menu[i]['kcal'])) + 'kcal\n'
        renderstring += '![' + menu[i]['name'] + '](' + menu[i]['thumbnailurl'] + ')\n'
        renderstring += '---\n'
    return renderstring

In [ ]:
def post(menu, url_hook):
    headers = {'Content-Type':'application/json'}
    data = {'text':'bot'}
    response = requests.post(url_hook, headers=headers, data=json.dumps({"text":menu}))

In [ ]:
def dailyUpdateAndPost(url_hook):
    post(getRenderString(updateMenu()), url_hook)

In [ ]:
def dailyMenuUpdate():
    url_hook_file = './url_hook.txt'
    with open(url_hook_file, 'rt') as f:
        url_hook = f.read()
    dailyUpdateAndPost(url_hook)

In [ ]:
if __name__ == '__main__':
    schedule.every().monday.at("02:00").do(dailyMenuUpdate)
    schedule.every().tuesday.at("02:00").do(dailyMenuUpdate)
    schedule.every().wednesday.at("02:00").do(dailyMenuUpdate)
    schedule.every().thursday.at("02:00").do(dailyMenuUpdate)
    schedule.every().friday.at("02:00").do(dailyMenuUpdate)
    while(True):
        schedule.run_pending()
        time.sleep(60)